In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [2]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary
import time

c:\Users\admin\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
import import_ipynb
from Datagen import Data

importing Jupyter notebook from Datagen.ipynb


In [4]:
def load_data():
    seglength = 9000
    train_ratio = 0.8
    val_ratio = 0.1

    focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio, seglength, class_num = 1)
    ige_train, ige_test, ige_train_labels, ige_test_labels = Data('D:\EEG\IGE', train_ratio, seglength, class_num = 2)
    
    X_train = torch.cat((focal_train, ige_train))
    Y_train = torch.cat((focal_train_labels, ige_train_labels)).view(-1, 1)
    X_test = torch.cat((focal_train, ige_train))
    Y_test = torch.cat((focal_train_labels, ige_train_labels)).view(-1, 1)

    del focal_train, focal_test, focal_train_labels, focal_test_labels, ige_train, ige_test, ige_train_labels, ige_test_labels

    pnes_train, pnes_test, pnes_train_labels, pnes_test_labels = Data('D:\EEG\PNES', train_ratio, seglength, class_num = 3)

    X_train = torch.cat((X_train, pnes_train))
    Y_train = torch.cat((Y_train, pnes_train_labels.view(-1, 1)))
    X_test = torch.cat((X_test, pnes_test))
    Y_test = torch.cat((Y_test, pnes_test_labels.view(-1, 1)))

    del pnes_train, pnes_test, pnes_train_labels, pnes_test_labels

    temporal_train, temporal_test, temporal_train_labels, temporal_test_labels = Data('D:\EEG\TLE', train_ratio, seglength, class_num = 4)

    X_train = torch.cat((X_train, temporal_train))
    Y_train = torch.cat((Y_train, temporal_train_labels.view(-1, 1)))
    X_test = torch.cat((X_test, temporal_test))
    Y_test = torch.cat((Y_test, temporal_test_labels.view(-1, 1)))

    del temporal_train, temporal_test, temporal_train_labels, temporal_test_labels

    X_val = X_train[:int(val_ratio * len(Y_train))]
    Y_val = Y_train[:int(val_ratio * len(Y_train))]
    X_train = X_train[int(val_ratio * len(Y_train)):]
    Y_train = Y_train[int(val_ratio * len(Y_train)):].view(-1, 1)

    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [5]:
#X_train, Y_train, X_val, Y_val, X_test, Y_test = load_data()

In [6]:

X_train = torch.randn([14231, 1, 9000, 18])
Y_train = torch.ones([14231, 1])
X_val = torch.randn([1581, 1, 9000, 18])
Y_val = torch.ones([1581, 1])
X_test = torch.randn([10249, 1, 9000, 18])
Y_test = torch.ones([10249, 1])


In [7]:
np.shape(Y_test)

torch.Size([10249, 1])

In [8]:
def evaluate(model, X, Y, params = ["fmeasure"]):
    torch.cuda.empty_cache()
    results = [] 
    predicted = []
    batch_size=1

    
    for i in range(int((len(X)/batch_size))):
        s = i*batch_size
        e = i*batch_size+batch_size
            
        inputs = Variable(X[s:e])
        pred = model(inputs)
            
        predicted.append(pred.data.cpu().numpy())
            
    predicted = torch.Tensor(predicted).view(-1, 1)

    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results



In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 9000

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 18))
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.padding1 = nn.ZeroPad2d(10)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 320, (5, 5), stride=5)
        self.batchnorm2 = nn.BatchNorm2d(320, False)

        # FC
        self.fc1 = nn.Linear(2309120, 1)

    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = self.padding1(x)
        # Layer 2
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)


        #x = x.permute(0, 3, 1, 2)



        x = x.reshape(-1, x.shape[1]* x.shape[2]* x.shape[3])
        x = F.sigmoid(self.fc1(x))
        return x



model = CNN().cuda()

#model.forward(X_train[0:200])
summary(model.cuda(), (1, 9000, 20))
#model.cpu().forward(X_train[0:2])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 16, 9000, 3]             304
       BatchNorm2d-2          [-1, 16, 9000, 3]              32
         ZeroPad2d-3         [-1, 16, 9020, 23]               0
            Conv2d-4         [-1, 320, 1804, 4]         128,320
       BatchNorm2d-5         [-1, 320, 1804, 4]             640
            Linear-6                    [-1, 1]       2,309,121
Total params: 2,438,417
Trainable params: 2,438,417
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.69
Forward/backward pass size (MB): 67.15
Params size (MB): 9.30
Estimated Total Size (MB): 77.14
----------------------------------------------------------------


In [10]:
batch_size = 160
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
torch.cuda.empty_cache()



for epoch in range(30):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(X_train[s:e].cuda())
        labels = Variable(Y_train[s:e].cuda())



        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        #print(loss.data)
    
    


Epoch  0

Epoch  1

Epoch  2

Epoch  3

Epoch  4

Epoch  5

Epoch  6

Epoch  7

Epoch  8

Epoch  9

Epoch  10

Epoch  11

Epoch  12

Epoch  13

Epoch  14

Epoch  15

Epoch  16

Epoch  17


In [ ]:
torch.cuda.empty_cache()

evaluate(model, X_val.cuda(), Y_val.cpu(), params = ["fmeasure"])

C:\Users\admin\AppData\Local\Temp\ipykernel_9916\1327882963.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  predicted = torch.Tensor(predicted).view(-1, 1)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
np.unique(predicted)

array([nan], dtype=float32)